In [15]:
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# load data
df = pd.read_parquet(
    r"https://github.com/ISL-0111/IDS701_Team_Project/raw/refs/heads/main/Data/cleaned_data.parquet"
)
df.sample(5)

,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_time,accept_gps_time,...,delivery_time,delivery_gps_time,delivery_gps_lng,delivery_gps_lat,ds,distance_km,accept_date,delivery_duration,delivery_duration_minutes,delivery_under_30
942814,3757269,61,Shanghai,3053,121.36807,31.34657,23436,1,2023-10-31 08:59:00,2023-10-31 08:59:00,...,2023-10-31 10:12:00,2023-10-31 10:12:00,121.36844,31.34638,1031,1.906986,2023-10-31,0 days 01:13:00,73.0,False
163104,1890415,8,Shanghai,191,121.52519,31.14764,9058,1,2023-09-30 11:11:00,2023-09-30 11:11:00,...,2023-09-30 13:03:00,2023-09-30 13:03:00,121.52558,31.14817,930,3.218530,2023-09-30,0 days 01:52:00,112.0,False
267631,1553144,16,Shanghai,3699,121.34429,31.40269,30108,14,2023-10-27 09:55:00,2023-10-27 09:55:00,...,2023-10-27 13:11:00,2023-10-27 13:11:00,121.34383,31.40220,1027,1.253373,2023-10-27,0 days 03:16:00,196.0,False
232223,1659176,9,Shanghai,4721,121.46369,30.90742,44573,1,2023-07-12 20:46:00,2023-07-12 20:46:00,...,2023-07-12 21:48:00,2023-07-12 21:48:00,121.46360,30.90759,712,2.234353,2023-07-12,0 days 01:02:00,62.0,False
815876,2597734,50,Shanghai,1991,121.47044,31.22071,13220,1,2023-10-05 08:40:00,2023-10-05 08:40:00,...,2023-10-05 09:15:00,2023-10-05 09:15:00,121.47064,31.22023,1005,3.180122,2023-10-05,0 days 00:35:00,35.0,False


In [7]:
print(df.shape)

(1469084, 23)


In [9]:
print(df.describe)

<bound method NDFrame.describe of          order_id  region_id      city  courier_id        lng       lat  \
0         3158819          1  Shanghai         164  121.52128  31.06614   
1          751342          1  Shanghai         164  121.52124  31.06687   
2         3380476          1  Shanghai         164  121.52106  31.06731   
3         2184571          1  Shanghai         164  121.52128  31.06616   
4          941371          1  Shanghai         164  121.52123  31.06614   
...           ...        ...       ...         ...        ...       ...   
1469079   3803282         93  Shanghai        2690  121.55562  31.07862   
1469080    960937         93  Shanghai        2690  121.55559  31.07807   
1469081   3894541         93  Shanghai        2690  121.55567  31.07864   
1469082   3946912         93  Shanghai        2690  121.55574  31.07873   
1469083   1320044         93  Shanghai        2690  121.55565  31.07866   

         aoi_id  aoi_type         accept_time     accept_gps_time

In [8]:
print(df.info)

<bound method DataFrame.info of          order_id  region_id      city  courier_id        lng       lat  \
0         3158819          1  Shanghai         164  121.52128  31.06614   
1          751342          1  Shanghai         164  121.52124  31.06687   
2         3380476          1  Shanghai         164  121.52106  31.06731   
3         2184571          1  Shanghai         164  121.52128  31.06616   
4          941371          1  Shanghai         164  121.52123  31.06614   
...           ...        ...       ...         ...        ...       ...   
1469079   3803282         93  Shanghai        2690  121.55562  31.07862   
1469080    960937         93  Shanghai        2690  121.55559  31.07807   
1469081   3894541         93  Shanghai        2690  121.55567  31.07864   
1469082   3946912         93  Shanghai        2690  121.55574  31.07873   
1469083   1320044         93  Shanghai        2690  121.55565  31.07866   

         aoi_id  aoi_type         accept_time     accept_gps_time  

In [6]:
# prepare dataset
df["accept_hour"] = df["accept_time"].dt.hour

In [11]:
# Filter Data for Top Regions
n_regions_under_consideration = 14
top_regions = df["region_id"].value_counts().head(n_regions_under_consideration).index
df_filtered = df[df["region_id"].isin(top_regions)]

# Keep only necessary columns
df_filtered = df_filtered[
    [
        "order_id",
        "region_id",
        "courier_id",
        "delivery_duration_minutes",
        "accept_date",
        "distance_km",
        "accept_hour",
    ]
]

print(df.shape)
df.sample(5)

(1469084, 23)


,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_time,accept_gps_time,...,delivery_gps_time,delivery_gps_lng,delivery_gps_lat,ds,distance_km,accept_date,delivery_duration,delivery_duration_minutes,delivery_under_30,accept_hour
1010192,209054,65,Shanghai,4252,121.42614,31.16266,26831,1,2023-08-09 18:09:00,2023-08-09 18:09:00,...,2023-08-09 18:30:00,121.42618,31.16260,809,1.857016,2023-08-09,0 days 00:21:00,21.0,True,18
787903,1447175,46,Shanghai,4837,121.56722,31.26474,27519,1,2023-06-19 21:37:00,2023-06-19 21:37:00,...,2023-06-19 22:28:00,121.56734,31.26492,619,1.169675,2023-06-19,0 days 00:51:00,51.0,False,21
1372314,1722806,87,Shanghai,2786,121.45691,31.22481,4117,0,2023-07-25 16:24:00,2023-07-25 16:24:00,...,2023-07-25 19:20:00,121.45675,31.22431,725,2.698517,2023-07-25,0 days 02:56:00,176.0,False,16
972691,1767243,63,Shanghai,1256,121.37551,31.16603,29057,1,2023-06-28 09:04:00,2023-06-28 09:04:00,...,2023-06-28 11:26:00,121.37566,31.16610,628,1.699861,2023-06-28,0 days 02:22:00,142.0,False,9
452827,4155040,29,Shanghai,3618,121.41026,31.24522,26350,0,2023-10-22 15:34:00,2023-10-22 15:34:00,...,2023-10-22 16:28:00,121.41063,31.24541,1022,1.174223,2023-10-22,0 days 00:54:00,54.0,False,15


In [12]:
# Create Daily Aggregations per Courier
daily_agg = (
    df_filtered.groupby(["courier_id", "accept_date"])
    .agg(
        task_count=("order_id", "nunique"),  # Number of tasks per day
        avg_delivery_duration_min=("delivery_duration_minutes", "mean"),
        avg_distance_km=("distance_km", "mean"),
        delivery_hour_mode=(
            "accept_hour",
            lambda x: x.mode()[0],
        ),  # Most frequent hour that day
    )
    .reset_index()
)

In [13]:
# --- Step 3: Create 'high_load' Flag ---
# Define high_load as days with task_count > median daily tasks across all couriers
median_daily_tasks = daily_agg["task_count"].median()
daily_agg["high_load"] = (daily_agg["task_count"] > median_daily_tasks).astype(int)

In [16]:
# --- Step 4: Run Regression ---
model = smf.ols(
    formula="np.log(avg_delivery_duration_min) ~ task_count * high_load + avg_distance_km + C(delivery_hour_mode)",
    data=daily_agg,
).fit(cov_type="HC3")

/Users/yirang/miniforge3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# --- Step 4: Run Regression ---
model = smf.ols(
    formula="np.log(avg_delivery_duration_min) ~ task_count * high_load + avg_distance_km + C(delivery_hour_mode)",
    data=daily_agg,
).fit(cov_type="HC3")

/Users/yirang/miniforge3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/yirang/miniforge3/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1733: RuntimeWarning: invalid value encountered in subtract
  return np.sum(weights * (model.endog - mean)**2)


ValueError: r_matrix performs f_test for using dimensions that are asymptotically non-normal

In [ ]:
# Step 5: Print Results ---
print(model.summary())

/Users/yirang/miniforge3/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1733: RuntimeWarning: invalid value encountered in subtract
  return np.sum(weights * (model.endog - mean)**2)


ValueError: r_matrix performs f_test for using dimensions that are asymptotically non-normal

In [ ]:
# --- Diagnostic Checks ---
# Check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = daily_agg[["task_count", "high_load", "avg_distance_km"]]  # Exclude categorical
X["interaction"] = X["task_count"] * X["high_load"]  # Add interaction term
X = sm.add_constant(X)
print(
    pd.DataFrame(
        {
            "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
            "feature": X.columns,
        }
    )
)

In [ ]:
# Check residuals
plt.scatter(model.fittedvalues, model.resid)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()
